In [1]:
import pandas as pd

In [11]:
# Load your final dataset (replace with your actual file path)
file1_path = "IHME-GBD_2021_DATA-5503ed84-1.csv"  # Risk factors
file2_path = "IHME-GBD_2021_DATA-6f8f7ceb-1.csv"  # Cause of death

file_air_pollution = "air pollution.csv"
file_co2 = "co2 Emission Africa.csv"
file_haq = "combined_haq_df.csv"

# Load datasets
df_risk = pd.read_csv(file1_path)
df_cause = pd.read_csv(file2_path)
df_air = pd.read_csv(file_air_pollution)
df_co2 = pd.read_csv(file_co2)
df_haq = pd.read_csv(file_haq)

# Load the datasets
air_pollution_df = pd.read_csv("air pollution.csv")
co2_emission_df = pd.read_csv("co2 Emission Africa.csv")

In [5]:
# Dataset 1: Health Burden (Cause of Death Only)
# drop the columns not needed.
df_health_burden = df_cause.drop(columns=['sex','age','upper', 'lower'])

#rename countries for uniformity
df_health_burden['location'] = df_health_burden['location'].replace({
    'Democratic Republic of the Congo': 'Democratic Republic of Congo',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'United Republic of Tanzania': 'Tanzania'
})

df_health_burden.head()

,measure,location,cause,metric,year,val
0,Deaths,Eritrea,Asthma,Number,2000,289.486429
1,Deaths,Eritrea,Interstitial lung disease and pulmonary sarcoi...,Number,2000,13.127577
2,Deaths,Eritrea,Other chronic respiratory diseases,Number,2000,37.860056
3,Deaths,Democratic Republic of Congo,Chronic obstructive pulmonary disease,Number,2001,7642.356241
4,Deaths,Senegal,Chronic obstructive pulmonary disease,Number,2003,860.605027


In [6]:
# Dataset 2: Risk Factor Burden
df_risk_factor = df_risk.drop(columns=['sex','age','upper', 'lower'])
df_risk_factor = df_risk_factor.rename(columns={'rei': 'risk_factor'})

#rename countries for uniformity
df_risk_factor['location'] = df_risk_factor['location'].replace({
    'Democratic Republic of the Congo': 'Democratic Republic of Congo',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'United Republic of Tanzania': 'Tanzania'
})


df_risk_factor.head()

,measure,location,cause,risk_factor,metric,year,val
0,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2000,257.840865
1,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2001,249.595886
2,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2002,245.426604
3,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2003,268.556783
4,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2004,298.192621


In [ ]:
# Dataset 3: Contextual Dataset
african_countries = [
    'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi',
    'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
    'Congo', 'Cote d’Ivoire', 'Democratic Republic of Congo', 'Djibouti',
    'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon',
    'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia',
    'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco',
    'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe',
    'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa',
    'South Sudan', 'Sudan', 'Togo', 'Tunisia', 'Uganda', 'Tanzania',
    'Zambia', 'Zimbabwe'
]


In [9]:
#filter the health access and quality index for african countries

# Filter the df_haq DataFrame for the required indicators and countries
# Apply filtering criteria
df_haq_filtered = df_haq[
    (df_haq['year_id'] >= 2000)
][['location_name', 'year_id', 'indicator_name', 'val']]

# Pivot the table to create separate columns for each indicator
df_haq_pivot = df_haq_filtered.pivot_table(
    index=['location_name', 'year_id'],
    columns='indicator_name',
    values='val',
    aggfunc='mean'  # Using mean in case of duplicates
).reset_index()

# Rename columns
df_haq_pivot = df_haq_pivot.rename(columns={
    'location_name': 'Country',
    'year_id': 'Year',
    'Chronic respiratory diseases': 'Chronic_Respiratory(RSD)',
    'HAQ Index': 'HAQ_Index'
})

# Display the final dataset
print(df_haq_pivot)

indicator_name   Country  Year  Chronic_Respiratory(RSD)  HAQ_Index
0                Algeria  2000                  1.180000  50.600000
1                Algeria  2005                  0.910000  55.300000
2                Algeria  2010                  0.734000  58.200000
3                Algeria  2015                  0.100000  63.700000
4                Algeria  2016                  0.560000  63.100000
..                   ...   ...                       ...        ...
319             Zimbabwe  2005                  1.659000  27.000000
320             Zimbabwe  2010                  1.954000  26.200000
321             Zimbabwe  2015                  0.500000  48.700000
322             Zimbabwe  2016                  1.703000  31.200000
323             Zimbabwe  2019                  0.000016  28.577096

[324 rows x 4 columns]


In [12]:
# Merge on 'Country' and 'Year'
df_air_pol = pd.merge(df_co2, df_air, on=["Country", "Year"], how="inner")

# Calculate Population Density
df_air_pol["Population Density"] = df_air_pol["Population"] / df_air_pol["Area (Km2)"]

# Select and reorder columns to match the final structure
final_columns_order = [
    "Country", "Sub-Region", "Year", "Population", "GDP PER CAPITA (USD)", "Area (Km2)",
    "Population Density", "Total CO2 Emission excluding LUCF (Mt)", "Nitrogen oxide",
    "Sulphur dioxide", "Carbon Monoxide", "NMVOCs", "Black Carbon", "Ammonia"
]

# Create the final DataFrame
df_air_pol = df_air_pol[final_columns_order]


df_air_pol.head()


,Country,Sub-Region,Year,Population,GDP PER CAPITA (USD),Area (Km2),Population Density,Total CO2 Emission excluding LUCF (Mt),Nitrogen oxide,Sulphur dioxide,Carbon Monoxide,NMVOCs,Black Carbon,Ammonia
0,Algeria,Northern Africa,2000,30774621,1780.38,2381741,12.921061,80.05,259742.44,99556.484,1163390.6,1002841.3,52766.760,82898.12
1,Algeria,Northern Africa,2001,31200985,1754.58,2381741,13.100075,78.65,259294.86,91087.230,1114166.4,978766.6,47093.310,83139.63
2,Algeria,Northern Africa,2002,31624696,1794.81,2381741,13.277974,82.40,293826.30,118613.930,1162100.8,959356.4,48631.273,83576.06
3,Algeria,Northern Africa,2003,32055883,2117.05,2381741,13.459013,88.19,289759.84,108086.620,1166238.5,1002011.7,49078.914,85245.90
4,Algeria,Northern Africa,2004,32510186,2624.80,2381741,13.649757,89.49,315098.84,121832.650,1199527.8,1053564.1,49500.914,88542.40


In [13]:
df_socioeconomic = pd.merge(df_air_pol, df_haq_pivot, on=['Country', 'Year'], how='left')
df_socioeconomic = df_socioeconomic.loc[:, ~df_socioeconomic.columns.str.contains('upper|lower')]


df_socioeconomic.head()

,Country,Sub-Region,Year,Population,GDP PER CAPITA (USD),Area (Km2),Population Density,Total CO2 Emission excluding LUCF (Mt),Nitrogen oxide,Sulphur dioxide,Carbon Monoxide,NMVOCs,Black Carbon,Ammonia,Chronic_Respiratory(RSD),HAQ_Index
0,Algeria,Northern Africa,2000,30774621,1780.38,2381741,12.921061,80.05,259742.44,99556.484,1163390.6,1002841.3,52766.760,82898.12,1.18,50.6
1,Algeria,Northern Africa,2001,31200985,1754.58,2381741,13.100075,78.65,259294.86,91087.230,1114166.4,978766.6,47093.310,83139.63,NaN,NaN
2,Algeria,Northern Africa,2002,31624696,1794.81,2381741,13.277974,82.40,293826.30,118613.930,1162100.8,959356.4,48631.273,83576.06,NaN,NaN
3,Algeria,Northern Africa,2003,32055883,2117.05,2381741,13.459013,88.19,289759.84,108086.620,1166238.5,1002011.7,49078.914,85245.90,NaN,NaN
4,Algeria,Northern Africa,2004,32510186,2624.80,2381741,13.649757,89.49,315098.84,121832.650,1199527.8,1053564.1,49500.914,88542.40,NaN,NaN


In [14]:
# Save all datasets to CSV
df_health_burden.to_csv("health_burden_dataset.csv", index=False)
df_risk_factor.to_csv("risk_factor_dataset.csv", index=False)
df_socioeconomic.to_csv("socioeconomic_dataset.csv", index=False)